In [1]:
import pyEX as p
import datetime
import os
import json

In [2]:
class Stock(object):
    
    def __init__(self):
        self.secret = 'sk_fae174660924464996b77d14209a973b'
        self.public = 'pk_2f3691af11ad4df583c95f2a4d89d44a'
        p.Client(api_token=self.secret, version='v1', api_limit=5)
    
    
    def get6Months(self,ticker):
        # 1 day
        history = p.chart(symbol, timeframe='6m', date=None, token=self.secret, version='v1', filter='')        
        PATH = f'./stock/6m/{ticker}.json'
        
        data = {}
        for chunk in history:
            formatted = {}
            formatted['open'] = chunk['open']
            formatted['close'] = chunk['close']
            formatted['high'] = chunk['high']
            formatted['low'] = chunk['low']
            formatted['volume'] = chunk['volume']
            
            data[chunk['date']] = formatted

        with open(PATH,'w') as JSON:
            json.dump(data,JSON)
            
        print('Done 6m.')
        
    
    def get5Days(self,ticker):
        # 10 min
        history = p.chart(symbol, timeframe='5dm', date=None, token=secret, version='v1', filter='')        
        PATH = f'./stock/5d/{ticker}.json'
        
        data = {}
        for chunk in history:
            formatted = {}
            formatted['price'] = chunk['average']
            formatted['volume'] = chunk['volume']

            date = chunk['date']
            try:
                data[date][chunk['minute']] = formatted
            except:
                data[date] = {chunk['minute']:formatted}
            
        with open(PATH,'w') as JSON:
            json.dump(data,JSON)
            
        print('Done 5d.')
    
    
    def getLiveQuote(self,ticker):
        # Snapshot
        today = p.quote(symbol=symbol, token=secret, version='v1')
        PATH = f'./stock/1d/{ticker}.json'
        
        old_data = {}
        if os.path.exists(PATH):
            with open(PATH,'r') as JSON:
                old_data = json.load(JSON)

        date = int(str(today['latestUpdate'])[0:-3])
        dateYYMMDD = str(datetime.datetime.fromtimestamp(date)).split(' ')[0]
        timestamp = str(datetime.datetime.fromtimestamp(date)).split(' ')[1]    
        
        formatted = {}
        formatted['price'] = today['iexRealtimePrice']
        formatted['volume'] = today['volume']
        
        try:
            volumeDelta = today['volume'] - old_data['lastVolume']
            formatted['volumeDelta'] = volumeDelta
        except:
            formatted['volumeDelta'] = 0
            
        old_data['lastVolume'] = today['volume']
        try:
            old_data[dateYYMMDD][timestamp] = formatted
        except:
            old_data[dateYYMMDD] = {timestamp:formatted}
        
        with open(PATH,'w') as JSON:
            json.dump(old_data,JSON)
            
        print('Done Live.')
    
    
    def run(self,ticker):
        PATH_6M = f'./stock/6m/{ticker}.json'
        PATH_5D = f'./stock/5d/{ticker}.json'
        PATH_1D = f'./stock/1d/{ticker}.json'
        
        if os.path.exists(PATH_6M) != True:
            self.get6Months(ticker)
        if os.path.exists(PATH_5D) != True:
            self.get5Days(ticker)
            
        self.getLiveQuote(ticker)

In [ ]:
c = Stock()
c.run('HEXO')